#  1. introduction

ce notebook a pour but d'explorer un jeu de données de santé, de générer des données synthétiques avec les modèles ctgan et tvae, et d’évaluer leur qualité.

l'objectif est aussi de tester la confidentialité des données synthétiques à l’aide de métriques avancées.


# 2. chargement des données

on commence par charger le fichier csv contenant les données réelles, puis on affiche les premières lignes pour voir à quoi ressemble le jeu de données.

on vérifiera ensuite les types de colonnes, les valeurs manquantes et les statistiques de base.

In [1]:
import os
import pandas as pd
base_dir = r"C:\Users\VictorAudoux\Desktop\synthetic_health_data\data"
chemin_fichier = os.path.join(base_dir, "effectifs.csv")
df = pd.read_csv(chemin_fichier, sep=";")


In [2]:
# 3. aperçu des données

# dimensions et types
print(f"dimensions du jeu de données : {df.shape}")
print("\ntypes de variables :")
print(df.dtypes)

# valeurs manquantes
print("\nvaleurs manquantes par colonne :")
print(df.isnull().sum())

# aperçu statistique des variables numériques
print("\nstatistiques descriptives :")
print(df.describe())


dimensions du jeu de données : (4636800, 16)

types de variables :
annee                   int64
patho_niv1             object
patho_niv2             object
patho_niv3             object
top                    object
cla_age_5              object
sexe                    int64
region                  int64
dept                   object
Ntop                  float64
Npop                    int64
prev                  float64
Niveau prioritaire     object
libelle_classe_age     object
libelle_sexe           object
tri                   float64
dtype: object

valeurs manquantes par colonne :
annee                       0
patho_niv1                  0
patho_niv2             483840
patho_niv3            1048320
top                         0
cla_age_5                   0
sexe                        0
region                      0
dept                        0
Ntop                  1238024
Npop                        0
prev                  1238024
Niveau prioritaire      60480
libelle_classe_

### 3. description et sélection des variables

ce jeu de données contient des informations agrégées sur des groupes de population selon différents critères : année, âge, sexe, région, pathologie, etc.

parmi les variables présentes :

- `annee`, `region`, `dept` : localisation et période  
- `sexe`, `cla_age_5` : caractéristiques démographiques  
- `patho_niv1` : catégorie principale de pathologie  
- `Npop` : nombre total de personnes dans le groupe  
- `Ntop` : nombre de personnes atteintes par la pathologie  
- `prev` : prévalence = proportion de personnes malades dans le groupe (`Ntop / Npop * 100`)

certaines colonnes comme `patho_niv2`, `patho_niv3` ou `libelle_*` non utilisées trop de valeurs manquantes

#### variables retenues

on garde les variables principales suivantes pour la suite :

- `annee`, `patho_niv1`, `cla_age_5`, `sexe`, `region`, `dept`, `prev`

les colonnes `Npop` et `Ntop` sont exclues car `prev` contient déjà l’information qu’elles décrivent sous forme de pourcentage.

on supprimera aussi les lignes où `prev` est manquant.


In [3]:

colonnes_utiles = ["annee", "patho_niv1", "cla_age_5", "sexe", "region", "dept", "prev"]

df_ = df[colonnes_utiles].copy()
df = df[df["prev"].notna()]

print(f"{len(df)} lignes restantes après filtrage.")
df.head()


3398776 lignes restantes après filtrage.


,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,Ntop,Npop,prev,Niveau prioritaire,libelle_classe_age,libelle_sexe,tri
0,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,28,999,6130.0,87700,6.986,"1,2,3",de 20 à 24 ans,hommes,3.0
1,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,32,62,3200.0,41610,7.686,"1,2,3",de 20 à 24 ans,hommes,3.0
2,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,32,80,1100.0,15770,6.974,"1,2,3",de 20 à 24 ans,hommes,3.0
3,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,44,51,1020.0,16260,6.285,"1,2,3",de 20 à 24 ans,hommes,3.0
4,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,44,57,1570.0,26040,6.029,"1,2,3",de 20 à 24 ans,hommes,3.0


In [4]:
df.dtypes


annee                   int64
patho_niv1             object
patho_niv2             object
patho_niv3             object
top                    object
cla_age_5              object
sexe                    int64
region                  int64
dept                   object
Ntop                  float64
Npop                    int64
prev                  float64
Niveau prioritaire     object
libelle_classe_age     object
libelle_sexe           object
tri                   float64
dtype: object

In [5]:
# répartitions simples
for col in ["annee", "sexe", "patho_niv1", "cla_age_5"]:
    print(f"\nDistribution de {col} :")
    print(df[col].value_counts(normalize=True).head())



Distribution de annee :
annee
2022    0.127311
2021    0.126937
2020    0.126215
2019    0.124655
2018    0.124301
Name: proportion, dtype: float64

Distribution de sexe :
sexe
9    0.361003
2    0.320935
1    0.318062
Name: proportion, dtype: float64

Distribution de patho_niv1 :
patho_niv1
Maladies cardioneurovasculaires                      0.194337
Cancers                                              0.164261
Maladies inflammatoires ou rares ou infection VIH    0.139365
Maladies neurologiques                               0.101748
Maladies psychiatriques                              0.101087
Name: proportion, dtype: float64

Distribution de cla_age_5 :
cla_age_5
tsage    0.063892
65-69    0.059637
60-64    0.059543
70-74    0.059164
55-59    0.059010
Name: proportion, dtype: float64


In [6]:

# on filtre les lignes avec sexe 9
df = df[df["sexe"].isin([1, 2])]

# on s'assure qu'il n'y a plus de valeurs manquantes
df = df.dropna()

# on affiche les dimensions finales
print(f"{len(df)} lignes conservées après nettoyage complet")
df.head()


1570168 lignes conservées après nettoyage complet


,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,Ntop,Npop,prev,Niveau prioritaire,libelle_classe_age,libelle_sexe,tri
0,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,28,999,6130.0,87700,6.986,"1,2,3",de 20 à 24 ans,hommes,3.0
1,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,32,62,3200.0,41610,7.686,"1,2,3",de 20 à 24 ans,hommes,3.0
2,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,32,80,1100.0,15770,6.974,"1,2,3",de 20 à 24 ans,hommes,3.0
3,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,44,51,1020.0,16260,6.285,"1,2,3",de 20 à 24 ans,hommes,3.0
4,2020,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,Hospitalisations hors pathologies repérées (av...,HHP_CAT_INC,20-24,1,44,57,1570.0,26040,6.029,"1,2,3",de 20 à 24 ans,hommes,3.0


In [7]:
from sklearn.preprocessing import QuantileTransformer
print("FINI")
# normalisation de la variable continue
qt = QuantileTransformer(output_distribution='normal', random_state=42)

# on crée une version normalisée de 'prev'
df["prev_norm"] = qt.fit_transform(df[["prev"]])

# vérification rapide
df[["prev", "prev_norm"]].describe()

colonnes_utiles = ["annee", "patho_niv1", "cla_age_5", "sexe", "region", "dept", "prev_norm"]
df_train = df[colonnes_utiles].copy()

colonnes_categorielle = ["annee", "patho_niv1", "cla_age_5", "sexe", "region", "dept"]

print("FINI")

FINI
FINI


In [ ]:
from ctgan import CTGAN
from tqdm import tqdm  # pour un affichage progressif si besoin (optionnel)

# création du modèle CTGAN avec affichage des logs
ctgan = CTGAN(epochs=300, verbose=True, log_frequency=True)

# entraînement du modèle sur les données
print("🔄 Entraînement du modèle CTGAN en cours...")
ctgan.fit(df_train, discrete_columns=colonnes_categorielle)
print("✅ Entraînement terminé.")

# génération de données synthétiques
print("📦 Génération des données synthétiques...")
synth_ctgan = ctgan.sample(10000)

# inversion de la normalisation sur la colonne 'prev'
synth_ctgan["prev"] = qt.inverse_transform(synth_ctgan[["prev_norm"]])
synth_ctgan.drop(columns=["prev_norm"], inplace=True)

print(f"✅ Données générées : {len(synth_ctgan)} lignes synthétiques")


🔄 Entraînement du modèle CTGAN en cours...


Gen. (-0.28) | Discrim. (-0.03):   4%|█▌                                        | 11/300 [1:49:17<53:08:13, 661.91s/it]

In [ ]:
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata

# création du metadata automatiquement à partir des données d'entraînement
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(df_train)

# création et entraînement du modèle TVAE
print("🔄 Entraînement du modèle TVAE en cours...")
tvae = TVAESynthesizer(metadata)
tvae.fit(df_train)
print("✅ Entraînement TVAE terminé.")

# génération des données synthétiques
print("📦 Génération des données synthétiques...")
synth_tvae = tvae.sample(10000)

# inversion de la normalisation sur la variable continue
synth_tvae["prev"] = qt.inverse_transform(synth_tvae[["prev_norm"]])
synth_tvae.drop(columns=["prev_norm"], inplace=True)

print(f"✅ Données générées : {len(synth_tvae)} lignes synthétiques")


## fusion des jeux de données synthétiques : ctgan + tvae

le modèle **ctgan** (conditional tabular GAN) est un réseau génératif conçu pour créer des données tabulaires synthétiques. il est particulièrement efficace pour modéliser les variables catégorielles complexes et les distributions déséquilibrées. il utilise un mécanisme de conditionnement qui lui permet de mieux capturer les relations entre les variables.

le modèle **tvae** (tabular variational autoencoder) repose sur un autre type d’architecture : les autoencodeurs variationnels. tvae est plus adapté aux distributions continues et offre souvent une meilleure couverture de l’espace des données (meilleure diversité), en générant des échantillons plus variés.

**combiner les deux modèles** permet de tirer profit de leurs forces respectives : ctgan produit souvent des données plus précises pour les variables catégorielles fréquentes, tandis que tvae génère une diversité plus large, utile pour couvrir les cas moins représentés. en fusionnant les deux jeux synthétiques (par exemple 40 % ctgan + 60 % tvae), on obtient un résultat plus équilibré, à la fois réaliste et riche en variations.


In [ ]:
# nombre total d'échantillons souhaité
n_total = 10000
n_ctgan = int(n_total * 0.4)
n_tvae = n_total - n_ctgan

# échantillonnage aléatoire (avec graine pour reproductibilité)
synth_fusion = pd.concat([
    synth_ctgan.sample(n=n_ctgan, random_state=42),
    synth_tvae.sample(n=n_tvae, random_state=42)
], ignore_index=True)

# aperçu
print(f"jeu fusionné : {len(synth_fusion)} lignes")
synth_fusion.head()
#shuffle
synth_fusion = synth_fusion.sample(frac=1, random_state=42).reset_index(drop=True)

